In [1]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import time
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
from scipy.io import loadmat
import nibabel as nib
import csv
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from scipy.ndimage.filters import convolve
from skimage import data, io, filters
import skimage
from skimage.morphology import convex_hull_image, erosion
from IPython import display
from scipy.ndimage import gaussian_filter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose,\
LeakyReLU, GaussianNoise, GlobalMaxPooling2D, ReLU, Input, Concatenate
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
from keras.models import Model
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

C:\Users\JYP\AppData\Local\Temp\ipykernel_21836\2458803103.py:25: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve


In [2]:
#PATH
Label = Path(r'../dataset/label/35label')
Image = Path(r'../dataset/original/35')

In [3]:
#IMAGE Path
Label_Path = list(Label.glob(r'**/*.jpg')) # glob : 파일을 찾아주는 함수
Image_Path = list(Image.glob(r'**/*.jpg'))

print('Label_Path : ', len(Label_Path))
print('Image_Path : ', len(Image_Path))

Label_Path :  13230
Image_Path :  6615


In [4]:
Label_Series = pd.Series(Label_Path, name='Label').astype(str) # Series : 1차원 배열
Image_Series = pd.Series(Image_Path, name='Image').astype(str)
print('Label_Series : ', len(Label_Series))
print('Image_Series : ', len(Image_Series))

Label_Series :  13230
Image_Series :  6615


In [5]:
print(Label_Series.head(-1))

0        ..\dataset\label\35label\ADNI_002_S_0685_18211...
1        ..\dataset\label\35label\ADNI_002_S_0685_18211...
2        ..\dataset\label\35label\ADNI_002_S_0685_18211...
3        ..\dataset\label\35label\ADNI_002_S_0685_18211...
4        ..\dataset\label\35label\ADNI_002_S_0685_18211...
                               ...                        
13224    ..\dataset\label\35label\ADNI_136_S_0429_10983...
13225    ..\dataset\label\35label\ADNI_136_S_0429_10983...
13226    ..\dataset\label\35label\ADNI_136_S_0429_10983...
13227    ..\dataset\label\35label\ADNI_136_S_0429_10983...
13228    ..\dataset\label\35label\ADNI_136_S_0429_10983...
Name: Label, Length: 13229, dtype: object


In [6]:
print(Image_Series.head(-1))

0       ..\dataset\original\35\ADNI_002_S_0685_18211_t...
1       ..\dataset\original\35\ADNI_002_S_0685_18211_t...
2       ..\dataset\original\35\ADNI_002_S_0685_18211_t...
3       ..\dataset\original\35\ADNI_002_S_0685_18211_t...
4       ..\dataset\original\35\ADNI_002_S_0685_18211_t...
                              ...                        
6609    ..\dataset\original\35\ADNI_136_S_0429_109839_...
6610    ..\dataset\original\35\ADNI_136_S_0429_109839_...
6611    ..\dataset\original\35\ADNI_136_S_0429_109839_...
6612    ..\dataset\original\35\ADNI_136_S_0429_109839_...
6613    ..\dataset\original\35\ADNI_136_S_0429_109839_...
Name: Image, Length: 6614, dtype: object


In [7]:
print('Label Path Len:', len(Label_Series[0]))
print('Image Path Len:', len(Image_Series[0]))

Label Path Len: 102
Image Path Len: 98


In [8]:
print('Label Path Example:\n', Label_Series[0])
print('Label Path Example:\n', Label_Series[6615])
print('Image Path Example:\n', Image_Series[0])

Label Path Example:
 ..\dataset\label\35label\ADNI_002_S_0685_18211\ADNI_002_S_0685_18211_L\ADNI_002_S_0685_18211_L_001.jpg
Label Path Example:
 ..\dataset\label\35label\ADNI_023_S_0061_132164_ACPC\ADNI_023_S_0061_132164_R\ADNI_023_S_0061_132164_R_001.jpg
Image Path Example:
 ..\dataset\original\35\ADNI_002_S_0685_18211_tal_noscale\ADNI_002_S_0685_18211_tal_noscale_001.jpg


In [18]:
#SPLITTING PROCESS
Split_Params_For_Label = '35label/'
Split_Params_For_Image = '35/'

L_Main_Path = r'../dataset/label/'
L_Target_Path = Label_Series[0].split(Split_Params_For_Label)
I_Main_Path = r'../dataset/original/'
I_Target_Path = Image_Series[0].split(Split_Params_For_Image)

print('L_Main_Path : ', L_Main_Path)
print('L_Target_Path : ', L_Target_Path)
print('I_Main_Path : ', I_Main_Path)
print('I_Target_Path : ', I_Target_Path)

L_Main_Path :  ../dataset/label/
L_Target_Path :  ['..\\dataset\\label\\35label\\ADNI_002_S_0685_18211\\ADNI_002_S_0685_18211_L\\ADNI_002_S_0685_18211_L_001.jpg']
I_Main_Path :  ../dataset/original/
I_Target_Path :  ['..\\dataset\\original\\35\\ADNI_002_S_0685_18211_tal_noscale\\ADNI_002_S_0685_18211_tal_noscale_001.jpg']


In [20]:
Common_Params = "/"

Label_Path_Before = L_Target_Path.split(Common_Params)

AttributeError: 'list' object has no attribute 'split'